## Импорт необходимых библиотек

In [ ]:
pip install mamba-ssm

  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 307.2/307.2 kB 4.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 44.6/44.6 kB 5.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 23.7/23.7 MB 38.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 823.6/823.6 kB 51.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 14.1/14.1 MB 68.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 731.7/731.7 MB 1.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 410.6/410.6 MB 3.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 121.6/121.6 MB 5.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 56.5/56.5 MB 8.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 124.2/124.2 MB 7.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 196.0/196.0 MB 6.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 166.0/166.0 MB 6

In [ ]:
from numpy import nan
import pandas as pd
import torch
import torch.nn as nn
import torch.optim as optim
from mamba_ssm import Mamba as SSM

from sklearn import preprocessing

# Функция для разделения модели на обучающий и тестовый наборы
from sklearn.model_selection import train_test_split

# Функция для оценки точности модели
from sklearn.metrics import accuracy_score

## Определение классов и функций

In [ ]:
class MambaTab(nn.Module):
    def __init__(self, input_dim, embedding_dim, hidden_dim, in_channels, out_channels, kernel_size):
        super(MambaTab, self).__init__()

        # Embedding Learner
        self.embedding = nn.Embedding(input_dim, embedding_dim, padding_idx=-1)#F.embedding(input_dim, embedding_dim)

        # Layer Normalization
        self.layer_norm = nn.LayerNorm(embedding_dim) #F.layer_norm(embedding_dim)

        # Linear Projections
        self.linear_projection = nn.Linear(embedding_dim, hidden_dim)

        # Convolution
        self.convolution = nn.Conv1d(in_channels, out_channels, kernel_size)

        # SiLU activation
        self.silu = nn.SiLU()

        # LeakyReLU activation
        self.leakyrelu = nn.LeakyReLU()

        # SSM (Structured State Space Mode)
        self.ssm = SSM(d_model=input_dim)  # Замените на соответствующий модуль

    def forward(self, x):
        # Embedding Learner
        embedded = self.embedding(x)

        # Layer Normalization
        normalized = self.layer_norm(embedded)

        # Linear Projections
        linear_1 = self.linear_projection(normalized)
        linear_2 = self.linear_projection(normalized)

        # Convolution
        conv_output = self.convolution(linear_1.permute(0, 2, 1))
        conv_output = conv_output.squeeze(2)

        # SiLU activation
        silu_output_1 = self.silu(conv_output)

        # SSM (Structured State Space Mode)
        ssm_output = self.ssm(silu_output_1)

        # LeakyReLU activation (Nonlinearity)
        leakyrelu_output = self.leakyrelu(ssm_output)

        # SiLU activation
        silu_output_2 = self.silu(linear_2)

        # Concatenate outputs
        # combined_output = torch.cat((silu_output_2, leakyrelu_output), dim=1)
        combined_output = silu_output_2 + leakyrelu_output

        linear_3 = self.linear_projection(combined_output)

        return torch.sigmoid(linear_3)

In [ ]:
def train_neural_network(model, X_train, y_train, optimizer, criterion, batch_size=32, num_epochs=10):
    model.train()

    for epoch in range(num_epochs):
        running_loss = 0.0
        correct_predictions = 0
        total_predictions = 0

        for i in range(0, len(X_train), batch_size):
            inputs = X_train[i:i+batch_size]
            labels = y_train[i:i+batch_size]

            optimizer.zero_grad()
            outputs = model(inputs)
            loss = criterion(outputs, labels)
            loss.backward()
            optimizer.step()

            running_loss += loss.item() * inputs.size(0)
            _, predicted = torch.max(outputs, 1)
            correct_predictions += (predicted == labels).sum().item()
            total_predictions += labels.size(0)

        epoch_loss = running_loss / len(X_train)
        epoch_accuracy = correct_predictions / total_predictions
        print(f"Epoch [{epoch+1}/{num_epochs}], Loss: {epoch_loss:.4f}, Accuracy: {epoch_accuracy:.4f}")

## Загрузка датасета

In [ ]:
DATASET_URL = 'https://archive.ics.uci.edu/ml/machine-learning-databases/adult/adult.data'
COLUMNS = [
    "age", "type_employer", "fnlwgt", "education", "education_num", "marital",
    "occupation", "relationship", "race","sex", "capital_gain", "capital_loss",
    "hr_per_week","country", "income"
]

In [ ]:
df = pd.read_csv(DATASET_URL, names=COLUMNS)
df

,age,type_employer,fnlwgt,education,education_num,marital,occupation,relationship,race,sex,capital_gain,capital_loss,hr_per_week,country,income
0,39,State-gov,77516,Bachelors,13,Never-married,Adm-clerical,Not-in-family,White,Male,2174,0,40,United-States,<=50K
1,50,Self-emp-not-inc,83311,Bachelors,13,Married-civ-spouse,Exec-managerial,Husband,White,Male,0,0,13,United-States,<=50K
2,38,Private,215646,HS-grad,9,Divorced,Handlers-cleaners,Not-in-family,White,Male,0,0,40,United-States,<=50K
3,53,Private,234721,11th,7,Married-civ-spouse,Handlers-cleaners,Husband,Black,Male,0,0,40,United-States,<=50K
4,28,Private,338409,Bachelors,13,Married-civ-spouse,Prof-specialty,Wife,Black,Female,0,0,40,Cuba,<=50K
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
32556,27,Private,257302,Assoc-acdm,12,Married-civ-spouse,Tech-support,Wife,White,Female,0,0,38,United-States,<=50K
32557,40,Private,154374,HS-grad,9,Married-civ-spouse,Machine-op-inspct,Husband,White,Male,0,0,40,United-States,>50K
32558,58,Private,151910,HS-grad,9,Widowed,Adm-clerical,Unmarried,White,Female,0,0,40,United-States,<=50K
32559,22,Private,201490,HS-grad,9,Never-married,Adm-clerical,Own-child,White,Male,0,0,20,United-States,<=50K


## Предобработка данных

In [ ]:
# Определение признаков и меток
X = df.drop(['income'], axis=1)
y = df['income']
# Разделяем данные на обучающий и тестовый наборы
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

In [ ]:
categorical = ['type_employer', 'education', 'marital', 'occupation', 'relationship', 'race', 'sex', 'country']
for feature in categorical:
        le = preprocessing.LabelEncoder()
        X_train[feature] = le.fit_transform(X_train[feature])
        X_test[feature] = le.transform(X_test[feature])

In [ ]:
scaler = preprocessing.StandardScaler()
X_train = pd.DataFrame(scaler.fit_transform(X_train), columns=X.columns)
X_test = pd.DataFrame(scaler.fit_transform(X_test), columns=X.columns)

In [ ]:
X_train = torch.LongTensor(X_train.to_numpy()) # .to('cuda')
X_test = torch.LongTensor(X_test.to_numpy()) # .to('cuda')

In [ ]:
X_train += abs(min(X_train.min(), X_test.min()))
X_test += abs(min(X_train.min(), X_test.min()))

In [ ]:
X_train

tensor([[4, 3, 4,  ..., 4, 4, 4],
        [4, 4, 4,  ..., 8, 4, 4],
        [5, 5, 4,  ..., 4, 4, 4],
        ...,
        [3, 4, 4,  ..., 4, 3, 4],
        [4, 5, 3,  ..., 4, 7, 4],
        [4, 4, 4,  ..., 4, 5, 4]])

In [ ]:
max(X_train.max(), X_test.max()).item()
X_train.max()
X_test.max()

tensor(16, device='cuda:0')

In [ ]:
indices = torch.tensor([-3, 5, -2, 0])

# Создание бинарных флагов
train_sign_flags = torch.where(indices < 0, torch.LongTensor([1]), torch.LongTensor([0]))

# Добавление дополнительного измерения к данным
X_train = torch.cat((X_train, train_sign_flags.unsqueeze(1)), dim=1)

## Создание и обучение модели

In [ ]:
# Пример использования модели
input_dim = 12 # max(X_train.max(), X_test.max()).item() + 1 # 18?
embedding_dim = len(df.columns) - 1 # Замените на размерность вложений
hidden_dim = 1 # Замените на размерность скрытого слоя
in_channels = 1 # Замените на количество входных каналов для свертки
out_channels = 1 # Замените на количество выходных каналов для свертки
kernel_size = 3 # Замените на размер ядра свертки

model = MambaTab(input_dim, embedding_dim, hidden_dim, in_channels, out_channels, kernel_size)
# device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
# model.to(device)

In [ ]:
print(model.testt(X_train))

In [ ]:
# Пример:
optimizer = optim.Adam(model.parameters(), lr=0.001)
criterion = nn.CrossEntropyLoss()
train_neural_network(model, X_train, y_train, optimizer, criterion, batch_size=32, num_epochs=10)

RuntimeError: Expected u.is_cuda() to be true, but got false.  (Could this error message be improved?  If so, please report an enhancement request to PyTorch.)